In [35]:


from random import random

class Neuron:
    def __init__(self, w = [], out = None, delta = 0.0):
        self.weights = w
        self.output = out
        self.delta = delta
    def __str__(self):
        return "weights: " + str(self.weights) + ", output: " + str(self.output) + ", delta: " + str(self.delta)
    def __repr__(self):
        return "weights: " + str(self.weights) + ", output: " + str(self.output) + ", delta: " + str(self.delta)



In [37]:
def activate(input, weights): 
    result = 0.0
    for i in range(0, len(input)):
        result += input[i] * weights[i]
    result += weights[len(input)]
    return result


#neuron transfer
from math import exp
def transfer(value):
    if (activationType == "identity"):
        return value
    elif (activationType == "Sigmoid"):
        return 1.0 / (1.0 + exp(-value))
    
#neuron computation/activation
def forwardPropagation(net, inputs):    
    for layer in net:
        newInputs = []
        for neuron in layer:
            activation = activate(inputs, neuron.weights)
            neuron.output = transfer(activation)
            newInputs.append(neuron.output)
        inputs = newInputs
    return inputs

#inverse transfer of a neuron
def transferInverse(val):
    if (activationType == "identity"):
        return val
    elif (activationType == "Sigmoid"):
        return val * ( 1 - val)

#error propagation
def backwardPropagation(net, expected):
    for i in range(len(net) - 1, 0, -1):
        crtLayer = net[i]
        errors = []
        if (i == len(net) - 1): #last layer
            for j in range(0, len(crtLayer)):
                crtNeuron = crtLayer[j]
                errors.append(expected[j] - crtNeuron.output)
        else:   #hidden layers
            for j in range(0, len(crtLayer)):
                crtError = 0.0
                nextLayer = net[i + 1]
                for neuron in nextLayer:
                    crtError += neuron.weights[j] * neuron.delta
                errors.append(crtError)
        for j in range(0, len(crtLayer)):
            crtLayer[j].delta = errors[j] * transferInverse(crtLayer[j].output)        
                        
#change the weights            
def updateWeights(net, example, learningRate):
    for i in range(0, len(net)):    #for each layer
        inputs = example[:-1]
        if (i > 0): #hidden layers or output layer
            inputs = [neuron.output for neuron in net[i - 1]]   #computed values of precedent layer
        for neuron in net[i]:   #update weight of all neurons of the current layer
            for j in range(len(inputs)):
                neuron.weights[j] += learningRate * neuron.delta * inputs[j]
            neuron.weights[-1] += learningRate * neuron.delta 




In [38]:
# network training
def trainingMLP(net, data, outputs, learningRate, noEpochs):
    for epoch in range(0, noEpochs):
        sumError = 0.0
        for inputs, expected in zip(data, outputs):
            computedOutputs = forwardPropagation(net, inputs)
            crtErr = sum([(expected[i] - computedOutputs[i]) ** 2 for i in range(0, len(expected))])
            sumError += crtErr
            backwardPropagation(net, expected)
            updateWeights(net, inputs, learningRate)
         
# network evaluation
def evaluatingMLP(net, data):
    computedOutputs = []
    for inputs in data:
        computedOutput = forwardPropagation(net, inputs[:-1])  
        computedOutputs.append(computedOutput[0])       
    return computedOutputs

def computePerformanceRegression(computedOutputs, realOutputs):
    error = sum([(computedOutputs[i] - realOutputs[i]) ** 2 for i in range(len(computedOutputs))])
    return error



In [43]:
PROBLEMTYPE = 'regression'

# f(x,y,z) = w0 + w1 * x + w2 * y + w3 * z 

regressionDataTrain = [[0.5, 0.05, 0.1, 0.95], 
                       [0.5, 0.1, 0.5, 0.6], 
                       [0.1, 0.2, 0.2, 0.2], 
                       [0.2, 0.3, 0.1, 0.6], 
                       [0.3, 0.3, 0.1, 0.8]]
regressionDataValidation = [[0.2, 0.4, 0.1, 0.7], 
                      [0.2, 0.3, 0.5, 0.2], 
                      [0.3, 0.3, 0.7, 0.2]]

import numpy as np 

inputTrain = np.array([line[:-1] for line in regressionDataTrain])
outputTrain = np.array([[line[-1]] for line in regressionDataTrain])
 
inputValidation = np.array([line[:-1] for line in regressionDataValidation])
outputValidation = np.array([[line[-1]] for line in regressionDataValidation])

activationType = 'identity'
# activationType = 'Sigmoid'
learningRate = 0.001
noEpochs = 100

noInputs = len(inputTrain[0])
noOutputs = 1

#initialisation of the weights for each neuron of all the layers (a hidden layer of 2 neurons and an output layer)
noHiddenNeurons = 2
net = []    

hiddenLayer = [Neuron([ random() for i in range(noInputs + 1)]) for h in range(noHiddenNeurons)]  
# hiddenLayer = [Neuron([ 0.0 for i in range(noInputs + 1)]) for h in range(noHiddenNeurons)]  
net.append(hiddenLayer)

outputLayer = [Neuron([ random() for i in range(noHiddenNeurons + 1)]) for o in range(noOutputs)]  
# outputLayer = [Neuron([ 0.0 for i in range(noHiddenNeurons + 1)]) for o in range(noOutputs)]  
net.append(outputLayer)


# network training
trainingMLP(net, inputTrain, outputTrain, learningRate, noEpochs)

# network testing 
computedOutputTrain = evaluatingMLP(net, inputTrain)
print("train SRE: ", computePerformanceRegression(computedOutputTrain, outputTrain))
        
computedOutputValidation = evaluatingMLP(net, inputValidation)
print("validation SRE: ", computePerformanceRegression(computedOutputValidation, outputValidation))



train SRE:  [0.78354864]
test SRE:  [1.68076653]


In [44]:
from sklearn import neural_network
ann = neural_network.MLPRegressor((2,), activation = activationType, solver = 'sgd', learning_rate_init = learningRate, max_iter = noEpochs)
ann.fit(inputTrain, outputTrain)
computedOutputs = ann.predict(inputTrain)
print("TOOL, train SRE: ", computePerformanceRegression(computedOutputs, outputTrain))    
        

computedOutputs = ann.predict(inputValidation)
print("TOOL, validation SRE: ", computePerformanceRegression(computedOutputs, outputValidation))
        

TOOL, train SRE:  [0.65770754]
TOOL, test SRE:  [0.17500994]
